# Check datat read speeds for numpy vs hdf5 files
April 22, 2021

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import sys
import os
import glob
import h5py

import time

5_Og=0.45_Sg=0.5_H0=100.0 \
22_Og=0.15_Sg=0.8_H0=100.0 \
24_Og=0.15_Sg=1.1_H0=100.0 \
26_Og=0.15_Sg=0.5_H0=100.0 

Nov 11, 2020 \
Fix H_0=70.0, Omega_m = 0.3 \
Sigma_8=0.5, 0.8, 1.1 

Nov 12, 2020 \
H_0=70.0 \
Omega_m= 0.15,0.3, Sigma_8 = 0.5,0.8, 1.1 


In [38]:
main_dir='/global/cfs/cdirs/m3363/vayyar/cosmogan_data/raw_data/3d_data/dataset1_smoothing_const_params_64cube_100k/'
# main_dir='/global/cscratch1/sd/vpa/proj/cosmogan/results_dir/128square/'
fname=main_dir+'norm_1_train_val.npy'

a1=np.load(fname,mmap_mode='r')[:10000]

In [39]:
a1.shape

(10000, 1, 64, 64, 64)

In [6]:
def f_samples_random(ip_arr,rank=0,num_ranks=1):
    '''
    Module to get part of the numpy image file
    '''
    
    data_size=ip_arr.shape[0]
    size=data_size//num_ranks
    
#     if gdict['batch_size']>size:
#         print("Caution: batchsize %s is greater than samples per GPU %s"%(gdict['batch_size'],size))
#         raise SystemExit
        
    ### Get a set of random indices from numpy array
    t1=time.time()
    idxs=np.arange(ip_arr.shape[0])
    t2=time.time()
    np.random.shuffle(idxs)
    t3=time.time()
    rnd_idxs=idxs[rank*(size):(rank+1)*size]
    t4=time.time()
    arr=ip_arr[rnd_idxs].copy()
    t5=time.time()
    
    print(t5-t4,t4-t3,t4-t2,t2-t1)
    return arr

In [40]:
# Time of reading slices

N=1000

## random indices

_=f_samples_random(a1[:N],0,1)

1.7952494621276855 1.6689300537109375e-06 6.67572021484375e-05 1.6927719116210938e-05


In [41]:
idxs=np.arange(N)
np.random.shuffle(idxs)
idxs


array([304, 583, 396, 974, 553, 743, 782, 778, 741, 514, 504, 134,  20,
       467, 472, 107, 748, 709, 944, 347,  78, 879, 546, 426, 211, 844,
       121, 750,  21, 541, 599, 574, 803, 163, 550, 231, 897, 166, 999,
       958, 286, 224, 158, 724, 105, 182, 649,  22, 593, 120, 812,  72,
       479, 135, 476, 594, 570, 391, 470, 669,  69, 332, 988, 930, 157,
       783, 288, 279, 964, 555, 164, 890, 630, 273, 818, 717, 529, 346,
       190, 170, 395, 691, 993, 563, 319, 315, 333, 640, 206, 616,  53,
       935, 155, 491, 350, 772, 696, 129, 730, 980,   1,  77, 527, 910,
       140, 781, 343, 791, 474, 813, 749, 950,  34, 639, 614, 377, 296,
       280, 790, 591, 241, 362, 183, 537, 815, 239, 880, 259, 345, 670,
       371, 381, 903, 774, 832, 159, 984, 970, 473, 620, 503, 522, 686,
       576, 863, 708, 448, 463, 499, 957, 849, 189,  93, 902, 307, 587,
       501, 664, 674, 446, 850, 747, 197, 336, 173, 117, 994, 763, 285,
       851, 992,  89, 316,  37, 534, 882, 161, 956, 989, 205, 25

In [42]:
%timeit a1[:N].copy()

759 ms ± 13 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [43]:
%%timeit 
a1[N:2*N].copy()


763 ms ± 8.04 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [44]:
%timeit a1[idxs].copy()

1.39 s ± 20.2 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)


In [49]:
# hdf5
fname=main_dir+'f.h5'
with h5py.File(fname,'w') as hf:
    hf.create_dataset("name_dataset",data=a1.copy())


In [52]:
t1=time.time()
with h5py.File(fname,'r') as hf:
    a2=hf['name_dataset'][:N]
t2=time.time()
print(t2-t1)
print(a2.shape)

0.677337646484375
(1000, 1, 64, 64, 64)


In [53]:
np.allclose(a1[:N],a2)

True

In [55]:
%%timeit 
with h5py.File(fname,'r') as hf:
    a2=hf['name_dataset'][:N]

552 ms ± 26.5 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
